# Cleaning
Daten laden und erkunden.

In [ ]:
import pandas as pd
from pathlib import Path

# Speicherort
base_path = Path.home() / "3_Sem" / "DS_Projekt1"

# CSV-Dateien laden
df_activity = pd.read_csv(base_path / "activity.csv")
df_training = pd.read_csv(base_path / "training_session.csv")
# df_target   = pd.read_csv(base_path / "training_target.csv")
# df_247      = pd.read_csv(base_path / "247ohr.csv")
# df_hr_daily = pd.read_csv(base_path / "df_hr_daily.csv")

# Spaltennamen ausgeben
print("\n activity.csv Spalten:")
print(df_activity.columns.tolist())

print("\n training_session.csv Spalten:")
print(df_training.columns.tolist())

# print("\n training_target.csv Spalten:")
# print(df_target.columns.tolist())

# print("\n 247ohr.csv Spalten:")
# print(df_247.columns.tolist())

# print("\n df_hr_daily.csv Spalten:")
# print(df_hr_daily.columns.tolist())


 activity.csv Spalten:
['filename', 'date', 'exportVersion', 'summary', 'samples', 'physicalInformation']

 training_session.csv Spalten:
['filename', 'date', 'exportVersion', 'name', 'deviceId', 'latitude', 'longitude', 'startTime', 'stopTime', 'timeZoneOffset', 'distance', 'duration', 'kiloCalories', 'physicalInformationSnapshot', 'exercises', 'maximumHeartRate', 'averageHeartRate', 'periodData', 'loadInformation', 'note', 'feeling']


## Feature auswahl
Welche Columns werden benötigt, um diese Frage zu beantworten? "Beinflusst das Training die Herzfrequenz"


In [2]:
df_activity['physicalInformation'][2]
# df_activity['summary'][2] # Level, Duration

"{'sex': 'FEMALE', 'birthday': '1995-04-19', 'height, cm': 168.0, 'weight, kg': 64.0, 'vo2Max': 44, 'maximumHeartRate': 197, 'sleepGoal': 'PT28800S'}"

In [3]:
df_training['date'][2]
df_training['startTime'][2]
df_training['stopTime'][2]
df_training['distance'][2]
df_training['duration'][2]
df_training['maximumHeartRate'][2]
df_training['averageHeartRate'][2]

np.float64(136.0)

### Activity

In [ ]:
import ast
import pandas as pd

df_activity = pd.read_csv("activity.csv")

rows = []
for _, row in df_activity.iterrows():
    try:
        summary = ast.literal_eval(row["summary"])
        physicalInformation = ast.literal_eval(row["physicalInformation"])  

        base = {
            "date": row["date"],
            "stepCount": summary.get("stepCount"),
            "stepsDistance": summary.get("stepsDistance"),
            "calories": summary.get("calories"),
            "sleepQuality": summary.get("sleepQuality"),
            "sleepDuration": summary.get("sleepDuration"),
            "inactivityAlertCount": summary.get("inactivityAlertCount"),
            "dailyMetMinutes": summary.get("dailyMetMinutes"),
            "maximumHeartRate": physicalInformation.get("maximumHeartRate"), 
        }

        # Aktivitätslevel in Dictionary umwandeln
        for lvl in summary.get("activityLevels", []):
            level_name = lvl["level"]
            duration = lvl["duration"]  # z. B. "PT7110S"

            # Sekunden aus "PTxxxxS" extrahieren
            seconds = int(duration.replace("PT", "").replace("S", "")) if "PT" in duration else 0
            base[level_name] = seconds

        rows.append(base)

    except Exception as e:
        print(f"Fehler bei Zeile: {e}")

df_activity_clean = pd.DataFrame(rows)
df_activity_clean["date"] = pd.to_datetime(df_activity_clean["date"], errors="coerce")

Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or string: nan
Fehler bei Zeile: malformed node or stri

In [ ]:
import re
import pandas as pd

df_activity_clean['date'] = pd.to_datetime(df_activity_clean['date'], errors='coerce').dt.date

# Sekundenwert aus "PTxxxxxS" extrahieren
df_activity_clean['sleepDuration_sec'] = (
    df_activity_clean['sleepDuration']
    .astype(str)
    .str.extract(r'PT(\d+)S')[0]
    .astype(float)
)

# In hh:mm:ss
df_activity_clean['sleepDuration'] = (
    pd.to_timedelta(df_activity_clean['sleepDuration_sec'], unit='s')
    .astype(str)
    .str.split().str[-1]
)

In [ ]:
import pandas as pd

# Sekunden zu hh:mm:ss
def sec_to_hms(x):
    try:
        return str(pd.to_timedelta(x, unit='s')).split()[-1]  # nimmt nur den Zeitteil
    except Exception:
        return None

# Umwandlung aller relevanten Spalten
cols = [
    'SLEEP', 'SEDENTARY', 'LIGHT', 'CONTINUOS_MODERATE',
    'INTERMITTENT_MODERATE', 'CONTINUOS_VIGOROUS',
    'INTERMITTENT_VIGOROUS', 'NON_WEAR', 'sleepDuration_sec'
]

for c in cols:
    df_activity_clean[c] = df_activity_clean[c].apply(sec_to_hms)

# Kontrolle
df_activity_clean

,date,stepCount,stepsDistance,calories,sleepQuality,sleepDuration,inactivityAlertCount,dailyMetMinutes,maximumHeartRate,NO_DATA,SLEEP,SEDENTARY,LIGHT,CONTINUOS_MODERATE,INTERMITTENT_MODERATE,CONTINUOS_VIGOROUS,INTERMITTENT_VIGOROUS,NON_WEAR,sleepDuration_sec
0,2024-01-26,14080,9755.738281,2547,0.872167,09:11:30,0,367.00,197.0,0,07:46:30,07:02:30,04:24:30,01:10:00,00:00:00,00:42:00,00:00:00,02:54:30,09:11:30
1,2025-06-21,25568,21580.402344,3181,0.000000,00:00:00,0,915.30,197.0,0,01:58:30,02:49:00,04:30:30,00:58:00,00:00:00,02:37:30,00:00:00,11:06:30,00:00:00
2,2021-09-16,11060,6132.661621,2651,0.885745,07:39:30,0,271.45,NaN,0,07:15:30,06:27:00,08:10:30,01:57:30,00:00:00,00:01:00,00:00:00,00:08:30,07:39:30
3,2024-07-15,27691,21221.250000,3294,0.850627,07:18:30,0,776.70,197.0,0,07:50:30,07:42:00,04:55:30,01:19:30,00:00:00,02:12:30,00:00:00,00:00:00,07:18:30
4,2021-11-05,436,196.745331,1439,0.835588,08:00:30,0,10.50,NaN,0,05:41:00,00:13:00,00:31:30,00:01:30,00:00:00,00:00:00,00:00:00,17:33:00,08:00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2025-03-26,259,182.312347,1381,0.000000,00:00:00,0,1.70,197.0,0,02:06:00,00:10:00,00:07:30,00:00:00,00:00:00,00:00:00,00:00:00,21:36:30,00:00:00
1712,2019-05-31,0,0.000000,1254,0.000000,00:00:00,0,0.00,NaN,0,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00
1713,2025-08-12,9862,4984.662109,2386,0.844280,07:52:00,0,216.15,197.0,0,08:45:30,05:59:30,07:31:00,01:02:00,00:00:00,00:15:00,00:00:00,00:27:00,07:52:00
1714,2021-06-17,16060,10128.861328,2596,0.906209,06:18:30,0,333.25,NaN,0,08:27:00,04:41:00,06:08:30,01:58:30,00:00:00,00:18:30,00:00:00,02:26:30,06:18:30


In [8]:
# df_activity_clean = df_activity_clean.drop(columns='sleepDuration_sec', inplace=True)
df_activity_clean.to_csv("activity_clean.csv", index=False)
df_activity_clean


,date,stepCount,stepsDistance,calories,sleepQuality,sleepDuration,inactivityAlertCount,dailyMetMinutes,maximumHeartRate,NO_DATA,SLEEP,SEDENTARY,LIGHT,CONTINUOS_MODERATE,INTERMITTENT_MODERATE,CONTINUOS_VIGOROUS,INTERMITTENT_VIGOROUS,NON_WEAR,sleepDuration_sec
0,2024-01-26,14080,9755.738281,2547,0.872167,09:11:30,0,367.00,197.0,0,07:46:30,07:02:30,04:24:30,01:10:00,00:00:00,00:42:00,00:00:00,02:54:30,09:11:30
1,2025-06-21,25568,21580.402344,3181,0.000000,00:00:00,0,915.30,197.0,0,01:58:30,02:49:00,04:30:30,00:58:00,00:00:00,02:37:30,00:00:00,11:06:30,00:00:00
2,2021-09-16,11060,6132.661621,2651,0.885745,07:39:30,0,271.45,NaN,0,07:15:30,06:27:00,08:10:30,01:57:30,00:00:00,00:01:00,00:00:00,00:08:30,07:39:30
3,2024-07-15,27691,21221.250000,3294,0.850627,07:18:30,0,776.70,197.0,0,07:50:30,07:42:00,04:55:30,01:19:30,00:00:00,02:12:30,00:00:00,00:00:00,07:18:30
4,2021-11-05,436,196.745331,1439,0.835588,08:00:30,0,10.50,NaN,0,05:41:00,00:13:00,00:31:30,00:01:30,00:00:00,00:00:00,00:00:00,17:33:00,08:00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2025-03-26,259,182.312347,1381,0.000000,00:00:00,0,1.70,197.0,0,02:06:00,00:10:00,00:07:30,00:00:00,00:00:00,00:00:00,00:00:00,21:36:30,00:00:00
1712,2019-05-31,0,0.000000,1254,0.000000,00:00:00,0,0.00,NaN,0,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00
1713,2025-08-12,9862,4984.662109,2386,0.844280,07:52:00,0,216.15,197.0,0,08:45:30,05:59:30,07:31:00,01:02:00,00:00:00,00:15:00,00:00:00,00:27:00,07:52:00
1714,2021-06-17,16060,10128.861328,2596,0.906209,06:18:30,0,333.25,NaN,0,08:27:00,04:41:00,06:08:30,01:58:30,00:00:00,00:18:30,00:00:00,02:26:30,06:18:30


### Training

In [9]:
df_training['date'][2] # Datum konvertieren
df_training['startTime'][2] # Datum und Zeit konvertieren
df_training['stopTime'][2] # Datum und Zeit konvertieren
df_training['distance'][2] 
df_training['duration'][2] # PT...S Format
df_training['kiloCalories'][2]
df_training['maximumHeartRate'][2]
df_training['averageHeartRate'][2]

np.float64(136.0)

In [ ]:
import pandas as pd
import re

# Datum
df_training['date'] = pd.to_datetime(df_training['date'], errors='coerce').dt.date

# Start- und Stopzeit
for col in ['startTime', 'stopTime']:
    df_training[col] = (
        pd.to_datetime(df_training[col], errors='coerce')
        .dt.strftime('%H:%M:%S')
    )

# Duration
df_training['duration_seconds'] = (
    df_training['duration']
    .astype(str)
    .str.extract(r'PT(\d+)S')[0]
    .astype(float)
)
df_training['duration'] = (
    pd.to_timedelta(df_training['duration_seconds'], unit='s')
    .astype(str)
    .str.split().str[-1]   # nur hh:mm:ss
)

# Numerische Spalten
for c in ['distance', 'kiloCalories', 'maximumHeartRate', 'averageHeartRate']:
    df_training[c] = pd.to_numeric(df_training[c], errors='coerce')

# Überflüssige Spalten löschen
cols_to_drop = [
    'filename', 'exportVersion', 'deviceId', 'physicalInformationSnapshot',
    'exercises', 'periodData', 'loadInformation', 'note', 'feeling'
]
df_training = df_training.drop(columns=[c for c in cols_to_drop if c in df_training.columns])

# Reihenfolge
order = [
    'date', 'startTime', 'stopTime', 'distance', 'duration',
    'kiloCalories', 'maximumHeartRate', 'averageHeartRate'
]
df_training = df_training[[c for c in order if c in df_training.columns]]

df_training

In [13]:
df_training.to_csv("training_clean.csv", index=False)
df_training_clean = pd.read_csv("training_clean.csv")

In [ ]:
df_247 = pd.read_csv(base_path / "247ohr.csv")
df_247.columns

Index(['filename', 'date_raw', 'deviceDays', 'date'], dtype='object')

## Herzfrequenz

In [6]:
import pandas as pd
import ast

# Datei laden
df_247 = pd.read_csv("247ohr.csv")

rows = []

# Jede Zeile kann mehrere 'deviceDays' enthalten
for _, row in df_247.iterrows():
    try:
        # JSON-ähnlichen String sicher in Python-Dict umwandeln
        device_days = ast.literal_eval(row["deviceDays"])

        # device_days ist eine Liste → iteriere über Einträge
        for day in device_days:
            date = day.get("date")
            time = day.get("time")


            # Jede Tagesstruktur enthält "samples"
            for sample in day.get("samples", []):
                rows.append({
                    "date": date,
                    "heartRate": sample.get("heartRate"),
                    "secondsFromDayStart": sample.get("secondsFromDayStart"),

                })
    except Exception as e:
        print("Fehler bei Zeile:", e)

# Neues flaches DataFrame mit Herzfrequenzen und Zeitpunkten
df_hr_247 = pd.DataFrame(rows)

# secondsFromDayStart → in Uhrzeit umrechnen (HH:MM:SS)
df_hr_247["time"] = pd.to_timedelta(df_hr_247["secondsFromDayStart"], unit="s").astype(str).str.split().str[-1]

# Datum in datetime konvertieren
df_hr_247["date"] = pd.to_datetime(df_hr_247["date"], errors="coerce")


# Kontrolle
df_hr_247.head()

,date,heartRate,secondsFromDayStart,time
0,2021-10-07,89,59576,16:32:56
1,2021-10-07,90,59577,16:32:57
2,2021-10-07,91,59578,16:32:58
3,2021-10-07,92,59579,16:32:59
4,2021-10-07,92,59580,16:33:00


In [12]:
df_hr_247.to_csv("df_247_clean.csv", index=False)
